# Movement Classification Model Training V2
This notebook walks through every step to train a machine learning model to classify human movements (e.g., squat, push-up) using joint angles.

In [2]:
import pandas as pd

In [3]:
angles_df = pd.read_csv('data/angles.csv')
labels_df = pd.read_csv('data/labels.csv')
dists_df = pd.read_csv('data/calculated_3d_distances.csv')
landmarks_df = pd.read_csv('data/landmarks.csv')


df = angles_df.merge(dists_df, on=['vid_id', 'frame_order']).merge(landmarks_df, on=['vid_id', 'frame_order']).merge(labels_df, on='vid_id')
df.head()


,vid_id,frame_order,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder,left_shoulder_left_wrist,...,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index,class
0,0,0,16.926802,7.667874,18.982162,112.747505,112.62553,112.099300,101.05565,44.616184,...,-4.885307,67.512770,40.333897,5.356711,73.934240,11.780330,-5.852993,73.78203,9.016774,jumping_jack
1,0,1,14.199318,8.954973,18.966124,109.707190,109.76263,110.645454,102.00027,44.785343,...,-4.753275,64.969570,45.439384,5.492989,73.177270,18.108229,-6.038326,72.70349,14.222010,jumping_jack
2,0,2,18.065800,10.315741,17.527954,114.562100,112.08965,113.340350,104.09502,44.907803,...,-4.517086,64.510980,48.996880,5.433758,72.199036,19.192911,-5.513490,71.79309,17.322145,jumping_jack
3,0,3,23.270214,17.336140,17.195545,117.674810,115.43172,114.634530,107.38297,45.922737,...,-4.674540,64.720245,53.581780,5.768750,72.696290,23.325266,-5.238461,72.11217,21.887375,jumping_jack
4,0,4,22.831680,13.822096,17.355429,117.536720,117.96766,112.306390,98.39078,40.620460,...,-4.098778,62.490230,52.845634,5.633003,70.438194,23.657516,-5.467475,70.08317,22.496626,jumping_jack


In [4]:
print(df.dtypes)
X = df.drop(columns=['vid_id', 'frame_order', 'class'])
y = df['class']

vid_id                                    int64
frame_order                               int64
right_elbow_right_shoulder_right_hip    float64
left_elbow_left_shoulder_left_hip       float64
right_knee_mid_hip_left_knee            float64
                                         ...   
z_left_foot_index                       float64
x_right_foot_index                      float64
y_right_foot_index                      float64
z_right_foot_index                      float64
class                                    object
Length: 125, dtype: object


In [5]:
df = pd.get_dummies(df, prefix="class",   

                                   drop_first=True)
print(df.dtypes) 



vid_id                                    int64
frame_order                               int64
right_elbow_right_shoulder_right_hip    float64
left_elbow_left_shoulder_left_hip       float64
right_knee_mid_hip_left_knee            float64
                                         ...   
z_right_foot_index                      float64
class_pull_up                              bool
class_push_up                              bool
class_situp                                bool
class_squat                                bool
Length: 128, dtype: object


In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
label_encoder.classes_

array(['jumping_jack', 'pull_up', 'push_up', 'situp', 'squat'],
      dtype=object)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [8]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf.fit(X_train, y_train)


from xgboost import XGBClassifier
clf = XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# from sklearn.neural_network import MLPClassifier
# clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
# clf.fit(X_train, y_train)




XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [9]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.9868334822758416
              precision    recall  f1-score   support

jumping_jack       0.99      0.99      0.99      3780
     pull_up       0.98      0.99      0.98      3835
     push_up       0.99      0.99      0.99      3472
       situp       0.99      0.99      0.99      3335
       squat       0.98      0.97      0.98      2363

    accuracy                           0.99     16785
   macro avg       0.99      0.99      0.99     16785
weighted avg       0.99      0.99      0.99     16785



In [10]:
import joblib

joblib.dump(clf, 'pose_classifier.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']